In [1]:
import scanpy as sc 
import numpy as np 
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit
from sklearn.preprocessing import LabelEncoder

In [2]:
def load_adata(file_path):
    adata = sc.read_h5ad(file_path)
    return adata 

def map_disease_labels(adata, label_column, encoder_column): 
    encoder = LabelEncoder()  # Create a single instance of LabelEncoder 
    adata.obs[encoder_column] = encoder.fit_transform(adata.obs[label_column].values)  # Transform labels 
    mapping_disease_label = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))  # Store mapping 
    return adata, mapping_disease_label 

In [3]:
path = r"../data/raw/COVID_Haniffa21/haniffa21.processed.h5ad" 
adata_covid = load_adata(path) 

In [4]:
gene_mask = adata_covid.var['feature_types'] == 'Gene Expression' 
adata_rna = adata_covid[:, gene_mask] 
adata_covid_binary = adata_rna[adata_rna.obs['Status'].isin(['Covid', 'Healthy'])] 

In [5]:
del adata_covid_binary.obsm 
del adata_covid_binary.uns

In [6]:
adata_covid_binary, mapping_disease_label_covid_binary = map_disease_labels(adata=adata_covid_binary, 
                                                                            label_column='Status', 
                                                                            encoder_column='disease_label_binary') 

In [7]:
print(adata_covid_binary.obs.groupby('disease_label_binary', observed=True)['patient_id'].nunique())

disease_label_binary
0    90
1    23
Name: patient_id, dtype: int64


In [8]:
adata_covid_binary.layers['preprocessed']= adata_covid_binary.X.copy()  # Save the preprocessed data in a new layer 
adata_covid_binary.layers['raw_counts'] = adata_covid_binary.layers['raw'].copy() # saving raw counts and using it for  preprocessing 
adata_covid_binary.X = adata_covid_binary.layers['raw_counts'].copy() 

In [9]:
adata_covid_binary.var["mt"] = adata_covid_binary.var_names.str.startswith("MT-") 
adata_covid_binary.var["ribo"] = adata_covid_binary.var_names.str.startswith(("RPS", "RPL")) 
adata_covid_binary.var["hb"] = adata_covid_binary.var_names.str.contains("^HB[^(P)]") 
sc.pp.calculate_qc_metrics(adata_covid_binary, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=False) 

In [10]:
patient_ids = adata_covid_binary.obs['patient_id'].unique()
adata_list = [adata_covid_binary[adata_covid_binary.obs['patient_id'] == pid].copy() for pid in patient_ids] 

In [11]:
# For each patient, get set of expressed genes (expressed in ≥1 cell)
expressed_genes_per_patient = []
for ad in adata_list:
    if hasattr(ad.X, "toarray"):
        expr = ad.X.toarray()
    else:
        expr = ad.X 
    gene_sum = expr.sum(axis=0)
    genes_expressed = set(ad.var_names[gene_sum > 0])
    expressed_genes_per_patient.append(genes_expressed)

# Intersection across all patients
common_genes = set.intersection(*expressed_genes_per_patient)
print(f"Number of genes expressed in all patients: {len(common_genes)}")

for i, ad in enumerate(adata_list):
    genes_to_keep = ad.var_names.isin(common_genes)
    adata_list[i] = ad[:, genes_to_keep].copy() 

Number of genes expressed in all patients: 12087


In [12]:
adata_list_filtered = [ad for ad in adata_list if ad.n_obs >= 1000]

Preprocessing using Groundgan

In [13]:
anndata = sc.concat(adata_list)
anndata.X.shape

(624325, 12087)

In [14]:
random_state=1000
import random
random.seed(random_state)

In [15]:
project_dir="/home/c01teaf/CISPA-az6/llm_tg-2024/GRouNdGAN"
data_path = f"{project_dir}/data/processed/COVID_Haniffa21-GGpp/CrossVal_{random_state}/"
print(data_path)

/home/c01teaf/CISPA-az6/llm_tg-2024/GRouNdGAN/data/processed/COVID_Haniffa21-GGpp/CrossVal_1000/


In [16]:
mkdir -p {data_path}

In [17]:
n_top_genes = 1000 # int(cfg.get("Preprocessing", "highly variable number"))
resolution = 0.15 # cfg.get("Preprocessing", "louvain res")
min_genes = 200 # int(cfg.get("Preprocessing", "min genes"))
min_cells = 3 # int(cfg.get("Preprocessing", "min cells"))
counts_per_cell_after = 20000 #int(cfg.get("Preprocessing", "library size"))
val_size = 1000 # int(cfg.get("Preprocessing", "validation set size"))
test_size = 1000 # int(cfg.get("Preprocessing", "test set size"))
path_to_train = f"{data_path}/train.h5ad" # cfg.get("Data", "train")
path_to_validation = f"{data_path}/validation.h5ad" # cfg.get("Data", "validation")
path_to_test = f"{data_path}/test.h5ad" # cfg.get("Data", "test")

In [18]:
test_size = int((anndata.obs.groupby('patient_id', observed=True)['patient_id'].size()*0.1).sum())
test_size

62432

In [19]:
anndata.n_obs

624325

In [20]:
original_order = np.arange(anndata.n_obs)  # Store the original cell order
np.random.shuffle(original_order)  # Shuffle the indices

# Apply the shuffled order to the AnnData object
anndata = anndata[original_order]

In [21]:
# clustering
ann_clustered = anndata.copy()
sc.pp.recipe_zheng17(ann_clustered,  n_top_genes=n_top_genes)
sc.tl.pca(ann_clustered, n_comps=50)
sc.pp.neighbors(ann_clustered, n_pcs=50)
sc.tl.louvain(ann_clustered, resolution=resolution)
anndata.obs["cluster"] = ann_clustered.obs["louvain"]

2025-11-07 18:02:22.382718: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2025-11-07 18:02:22.382765: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Trying to set attribute `.obs` of view, copying.


In [22]:
from collections import Counter

# get cluster ratios
cells_per_cluster = Counter(anndata.obs["cluster"])
cluster_ratios = dict()
for key, value in cells_per_cluster.items():
    cluster_ratios[key] = value / anndata.shape[0]
anndata.uns["cluster_ratios"] = cluster_ratios
anndata.uns["clusters_no"] = len(cluster_ratios)

In [23]:
# filtering
sc.pp.filter_cells(anndata, min_genes=min_genes)
sc.pp.filter_genes(anndata, min_cells=min_cells)
anndata.uns["cells_no"] = anndata.shape[0]
anndata.uns["genes_no"] = anndata.shape[1]

In [24]:
canndata = anndata.copy()

In [25]:
# library-size normalization
sc.pp.normalize_per_cell(canndata, counts_per_cell_after=counts_per_cell_after)

In [26]:
# identify highly variable genes
from scipy.sparse import issparse


sc.pp.log1p(canndata)  # logarithmize the data
sc.pp.highly_variable_genes(canndata, n_top_genes=n_top_genes)

if issparse(canndata.X):
    canndata.X = np.exp(canndata.X.toarray()) - 1  # get back original data
else:
    canndata.X = np.exp(canndata.X) - 1  # get back original data

anndata = anndata[
    :, canndata.var["highly_variable"]
]  # only keep highly variable genes

sc.pp.normalize_per_cell(anndata, counts_per_cell_after=counts_per_cell_after)

Trying to set attribute `.obs` of view, copying.


In [27]:
# sort genes by name (not needed)
sorted_genes = np.sort(anndata.var_names)
anndata = anndata[:, sorted_genes]

In [28]:
from sklearn.model_selection import train_test_split
import numpy as np

def split_data(anndata, label_column):
    y = anndata.obs[label_column].values
    X_idx = np.arange(anndata.n_obs)
    X_train, X_temp, y_train, y_temp = train_test_split(
                                        X_idx, 
                                        y, 
                                        test_size=test_size+val_size,
                                        random_state=random_state,
                                        stratify=y
                                    )
    X_val, X_test, y_val, y_test = train_test_split(
                                    X_temp, 
                                    y_temp,
                                    test_size=test_size,
                                    random_state=random_state,
                                    stratify=y_temp
                                )
    anndata_train = anndata[X_train].copy()
    anndata_val = anndata[X_val].copy()
    anndata_test = anndata[X_test].copy()
    return anndata_train, anndata_val, anndata_test

In [29]:
stratified_sampling = True

In [30]:
if stratified_sampling:
    label_column='Status'
    anndata_train, anndata_val, anndata_test = split_data(anndata, label_column)
else:
    anndata_val = anndata[:val_size]
    anndata_test = anndata[val_size : test_size + val_size]
    anndata_train = anndata[test_size + val_size :]

In [31]:
anndata_train_covid = anndata_train[anndata_train.obs.disease_label_binary == 0]
anndata_train_healthy = anndata_train[anndata_train.obs.disease_label_binary == 1]

anndata_test_covid = anndata_test[anndata_test.obs.disease_label_binary == 0]
anndata_test_healthy = anndata_test[anndata_test.obs.disease_label_binary == 1]


anndata_val_covid = anndata_val[anndata_val.obs.disease_label_binary == 0]
anndata_val_healthy = anndata_val[anndata_val.obs.disease_label_binary == 1]

In [32]:
for split in [anndata_train, anndata_val, anndata_test]:
    print(split.obs[label_column].value_counts(normalize=True))
    print("---")

Covid      0.844566
Healthy    0.155434
Name: Status, dtype: float64
---
Covid      0.845
Healthy    0.155
Name: Status, dtype: float64
---
Covid      0.844551
Healthy    0.155449
Name: Status, dtype: float64
---


In [33]:
for split in [anndata_train_covid, anndata_val_covid, anndata_test_covid]:
    print(len(split.obs['patient_id'].unique()))
    print("---")

90
---
89
---
90
---


In [34]:
for split in [anndata_train_healthy, anndata_val_healthy, anndata_test_healthy]:
    print(len(split.obs['patient_id'].unique()))
    print("---")

23
---
21
---
23
---


In [35]:
for split in [anndata_train, anndata_val, anndata_test]:
    print(len(split.obs['initial_clustering'].unique()))
    print(split.obs['initial_clustering'].value_counts())
    print("---")

18
CD4             127072
CD14            108861
CD8              86422
NK_16hi          81211
B_cell           56208
CD16             18311
Platelets        14678
Treg             11550
NK_56hi           9962
gdT               9244
Plasmablast       8850
Lymph_prolif      6271
DCs               6263
MAIT              5862
pDC               4269
HSC               2892
RBC               2367
Mono_prolif        581
Name: initial_clustering, dtype: int64
---
18
CD4             235
CD14            214
CD8             159
NK_16hi         143
B_cell           86
CD16             31
Platelets        26
Treg             23
NK_56hi          17
gdT              14
DCs              13
Plasmablast      10
Lymph_prolif     10
MAIT              6
HSC               5
pDC               5
RBC               2
Mono_prolif       1
Name: initial_clustering, dtype: int64
---
18
CD4             14169
CD14            12208
CD8              9711
NK_16hi          8855
B_cell           6274
CD16             2061

In [36]:
for split in [anndata_train_covid, anndata_val_covid, anndata_test_covid]:
    print(len(split.obs['initial_clustering'].unique()))
    print(split.obs['initial_clustering'].value_counts())
    print("---")

18
CD4             101718
CD14             99637
NK_16hi          69815
CD8              69545
B_cell           49371
CD16             15197
Platelets        12749
Treg              9440
Plasmablast       8741
NK_56hi           7965
Lymph_prolif      6006
gdT               5998
DCs               4370
MAIT              4076
pDC               3648
HSC               2765
RBC               2076
Mono_prolif        578
Name: initial_clustering, dtype: int64
---
18
CD14            194
CD4             182
CD8             128
NK_16hi         123
B_cell           77
CD16             30
Platelets        22
Treg             20
NK_56hi          15
Plasmablast      10
gdT              10
Lymph_prolif      8
DCs               8
MAIT              6
HSC               5
pDC               4
RBC               2
Mono_prolif       1
Name: initial_clustering, dtype: int64
---
18
CD4             11308
CD14            11140
CD8              7823
NK_16hi          7631
B_cell           5552
CD16             1710

In [37]:
for split in [anndata_train_healthy, anndata_val_healthy, anndata_test_healthy]:
    print(len(split.obs['initial_clustering'].unique()))
    print(split.obs['initial_clustering'].value_counts())
    print("---")

18
CD4             25354
CD8             16877
NK_16hi         11396
CD14             9224
B_cell           6837
gdT              3246
CD16             3114
Treg             2110
NK_56hi          1997
Platelets        1929
DCs              1893
MAIT             1786
pDC               621
RBC               291
Lymph_prolif      265
HSC               127
Plasmablast       109
Mono_prolif         3
Name: initial_clustering, dtype: int64
---
13
CD4             53
CD8             31
NK_16hi         20
CD14            20
B_cell           9
DCs              5
Platelets        4
gdT              4
Treg             3
Lymph_prolif     2
NK_56hi          2
CD16             1
pDC              1
Name: initial_clustering, dtype: int64
---
18
CD4             2861
CD8             1888
NK_16hi         1224
CD14            1068
B_cell           722
CD16             351
gdT              342
MAIT             228
Treg             227
NK_56hi          223
DCs              217
Platelets        212
pDC       

In [38]:
print(len(anndata_train_covid), len(anndata_test_covid))

473695 52727


In [39]:
print(len(anndata_train_healthy), len(anndata_test_healthy))

87179 9705


In [40]:
project_path="/home/c01teaf/CISPA-az6/llm_tg-2024/GRouNdGAN"

In [41]:
import os

In [42]:
tag='COVID_Haniffa21-GGpp-v2'
os.makedirs(f"{project_path}/data/processed/{tag}/CrossVal_{random_state}", exist_ok=True)

anndata_train.write_h5ad(f"{project_path}/data/processed/{tag}/CrossVal_{random_state}/train.h5ad")
anndata_val.write_h5ad(f"{project_path}/data/processed/{tag}/CrossVal_{random_state}/validation.h5ad")
anndata_test.write_h5ad(f"{project_path}/data/processed/{tag}/CrossVal_{random_state}/test.h5ad")
    
####
print("Successfully preprocessed and and saved dataset")

/usr/local/lib/python3.9/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'sample_id' as categorical
/usr/local/lib/python3.9/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'full_clustering' as categorical
/usr/local/lib/python3.9/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will a

Successfully preprocessed and and saved dataset


In [43]:
tag='COVID_Haniffa21-GGpp-covid-v2'
os.makedirs(f"{project_path}/data/processed/{tag}/CrossVal_{random_state}", exist_ok=True)

anndata_train_covid.write_h5ad(f"{project_path}/data/processed/{tag}/CrossVal_{random_state}/train.h5ad")
anndata_val_covid.write_h5ad(f"{project_path}/data/processed/{tag}/CrossVal_{random_state}/validation.h5ad")
anndata_test_covid.write_h5ad(f"{project_path}/data/processed/{tag}/CrossVal_{random_state}/test.h5ad")
    
####
print("Successfully preprocessed and and saved dataset")

/usr/local/lib/python3.9/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
/usr/local/lib/python3.9/site-packages/anndata/_core/anndata.py:1236: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'sample_id' as categorical
/usr/local/lib/python3.9/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
Trying to set attribute `.obs` of view, copying.
... storing 'full_clustering' as categor

Successfully preprocessed and and saved dataset


In [44]:
tag='COVID_Haniffa21-GGpp-healthy-v2'
os.makedirs(f"{project_path}/data/processed/{tag}/CrossVal_{random_state}", exist_ok=True)
anndata_train_healthy.write_h5ad(f"{project_path}/data/processed/{tag}/CrossVal_{random_state}/train.h5ad")
anndata_val_healthy.write_h5ad(f"{project_path}/data/processed/{tag}/CrossVal_{random_state}/validation.h5ad")
anndata_test_healthy.write_h5ad(f"{project_path}/data/processed/{tag}/CrossVal_{random_state}/test.h5ad")
    
####
print("Successfully preprocessed and and saved dataset")

/usr/local/lib/python3.9/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
/usr/local/lib/python3.9/site-packages/anndata/_core/anndata.py:1236: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'sample_id' as categorical
/usr/local/lib/python3.9/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
Trying to set attribute `.obs` of view, copying.
... storing 'full_clustering' as categor

Successfully preprocessed and and saved dataset


In [45]:
ls {project_dir}/data/processed/COVID_Haniffa21-GGpp-covid-v2/CrossVal_1000

test.h5ad  train.h5ad  validation.h5ad


In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [47]:
X_train = anndata_train.X
y_train = anndata_train.obs['Status']

In [48]:
model = LogisticRegression(max_iter=1000)  # increase iterations if needed
model.fit(X_train, y_train)

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

In [49]:
X_test = anndata_test.X
y_test = anndata_test.obs['Status']

In [50]:
y_pred = model.predict(X_test)

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Confusion Matrix:
 [[50692  2035]
 [ 4684  5021]]

Classification Report:
               precision    recall  f1-score   support

       Covid       0.92      0.96      0.94     52727
     Healthy       0.71      0.52      0.60      9705

    accuracy                           0.89     62432
   macro avg       0.81      0.74      0.77     62432
weighted avg       0.88      0.89      0.89     62432



In [51]:
X_val = anndata_val.X
y_val = anndata_val.obs['Status']

In [52]:
y_pred_val = model.predict(X_val)

print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred_val))
print("\nClassification Report:\n", classification_report(y_val, y_pred_val))

Confusion Matrix:
 [[810  35]
 [ 74  81]]

Classification Report:
               precision    recall  f1-score   support

       Covid       0.92      0.96      0.94       845
     Healthy       0.70      0.52      0.60       155

    accuracy                           0.89      1000
   macro avg       0.81      0.74      0.77      1000
weighted avg       0.88      0.89      0.88      1000

